In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import pipeline
import torch
import tensorflow as tf
print("x")


In [ ]:
models = [
    "lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    "finiteautomata/beto-sentiment-analysis",
    "cardiffnlp/twitter-roberta-base-sentiment",
    "cardiffnlp/twitter-roberta-base-sentiment-latest",
    "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual",
    "Cloudy1225/stackoverflow-roberta-base-sentiment",
    "nlptown/bert-base-multilingual-uncased-sentiment",
    "Moritz-Pfeifer/CentralBankRoBERTa-sentiment-classifier",
    "siebert/sentiment-roberta-large-english",
]
print("X")


In [ ]:
import pandas as pd

df = pd.read_json("/kaggle/input/f1-comms/dataset_with_drivers.json")
df.head(10)

In [ ]:
from datasets import Dataset

mapping = {
        "LABEL_0": "negative",
        "LABEL_1": "neutral",
        "LABEL_2": "positive",
        "1 star": "negative",
        "2 stars": "negative",
        "3 stars": "neutral",
        "4 stars": "positive",
        "5 stars": "positive",
        "NEU": "neutral",
        "POS": "positive",
        "NEG": "negative",
    }
def get_sentiment(text: str, model):
    sentiment_label = model(text)[0]["label"]
    return mapping.get(sentiment_label, sentiment_label)

print("X")

In [ ]:
for model_name in models:
    # Load the pipeline
    sentiment_pipeline = pipeline("text-classification", model=model_name, device=0, truncation=True, max_length=512)
    
    hf_dataset = Dataset.from_pandas(df)

    def compute_sentiment(batch):
        results = sentiment_pipeline(batch["text"], batch_size=32)  # Adjust batch size based on GPU memory
        batch["sentiment"] = [mapping.get(result["label"], result["label"]) for result in results]
        return batch

    hf_dataset = hf_dataset.map(compute_sentiment, batched=True, batch_size=32)

    col_name = f"sentiment_{model_name}"
    df[col_name] = hf_dataset["sentiment"]
    print(df.head(5))

In [ ]:
df.to_json("dataset_ready_to_train.json", orient="records")